In [30]:
import sys
sys.path.append("/home/user/relax/python")

Обучение средствами tvm примитивной сети для распознавания рукописных цифр датасета MNIST.

In [31]:
import tvm
from tvm.relay import Call
from tvm import relax, tir
from tvm.relax.testing import nn
from tvm.script import relax as R
import numpy as np

import pandas as pd

from tvm.relax.training import SetupTrainer
from tvm.relax.training.trainer import Trainer
from tvm.script.parser import ir as I
from tvm.relax.training.optimizer import SGD, Adam
from tvm.relax.training.loss import MSELoss, CrossEntropyLoss, L1Loss

In [32]:
import tvm.testing
import numpy as np
from tvm import relax
from tvm.script.parser import ir as I
from tvm.relax.training.optimizer import SGD
from tvm.relax.training.loss import MSELoss
from tvm.relax.training.trainer import Trainer

1. Создаем модель, содержащую два полносвязных слоя. Модель принимает на вход 1 изобранжение в формате (1, 784) и возвращает вектор вероятностей для каждой цифры (1, 10).

In [33]:
@I.ir_module
class MLP:
    @R.function
    def predict(
        w0: R.Tensor((784, 512), "float32"),
        b0: R.Tensor((1, 512), "float32"),
        w1: R.Tensor((512, 10), "float32"),
        b1: R.Tensor((1, 10), "float32"),
        x: R.Tensor((1, 784), "float32")
    ):
        with R.dataflow():
            lv0 = R.matmul(x, w0)
            lv1 = R.add(lv0, b0)
            lv2 = R.nn.relu(lv1)
            lv3 = R.matmul(lv2, w1)
            lv4 = R.add(lv3, b1)
            out = R.nn.relu(lv4)
            R.output(out)
        return out

2. Загружаем датасет MNIST из keras.

In [36]:
from keras.datasets import mnist
from keras.utils import to_categorical

dtype = "float32"
target_type = "float32"
(X_train, Y_train), (_, _) = mnist.load_data()
X_train = X_train.reshape((60000, 1, 28 *28))
X_train = X_train.astype(dtype) / 255
Y_train = to_categorical(Y_train)
dataset = []

for i in range(60000):
    image = X_train[i]
    label = Y_train[i]
    label = label.reshape(10,)
    dataset.append([image, np.array([label]).astype(target_type)])

3. Тренировка модели.
Задаем setup_trainer c функцией потерь MSE и оптимизатором SGD.
Результат обучения будет сравнивать с аналогичной моделью, обученной с помощью Keras, при обучении которой использовалась функция потерь categorical crossentropy, которая предпочтительнее для решаемой задачи, однако на данный момент в tvm успешно отработывает только MSELoss.
Тренируем модель 10 эпох. Обучение заняло 20 минут.
Время обучения модели с помощью keras на 10 эпохах составило 20 секунд.

### SGD = 0.01, MSELoss sum

In [42]:
pred_sinfo = relax.TensorStructInfo((1, 10), "float32")
setup_trainer = SetupTrainer(
    MSELoss(reduction="sum"),
    SGD(0.01),
    [pred_sinfo, pred_sinfo],
)
params_num = 4
trainer = Trainer(MLP, params_num, setup_trainer)
trainer.build("llvm", tvm.cpu(0))
trainer.xaiver_uniform_init_params()

last_loss = np.inf
for epoch in range(10):
    loss = 0
    batch_size = 60000
    for i in range(batch_size):
        loss += trainer.update_params(dataset[i][0], dataset[i][1]).numpy()
    print("#epoch", epoch, "loss=", loss / batch_size)
trainer.predict(dataset[0][0])

#epoch 0 loss= 0.4149585630603538
#epoch 1 loss= 0.2773886393406354
#epoch 2 loss= 0.14383645315356663
#epoch 3 loss= 0.13113220750746007
#epoch 4 loss= 0.12444584900149694
#epoch 5 loss= 0.11981310628275013
#epoch 6 loss= 0.09427826497765197
#epoch 7 loss= 0.028874767015046578
#epoch 8 loss= 0.022517451559328952
#epoch 9 loss= 0.018696302398676836


<tvm.nd.NDArray shape=(1, 10), cpu(0)>
array([[0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 1.0235652, 0.       , 0.       ]], dtype=float32)

4. Определяем функцию оценки модели. Используется метрика Accuracy.

In [43]:
from sklearn.metrics import accuracy_score
from keras.datasets import mnist

def eval(_trainer):
    (_, _), (X_test, Y_test) = mnist.load_data()
    predicted=[]
    target=[]
    X_test = X_test.astype(dtype) / 255
    for i in range(10000):
        image = X_test[i]
        label = Y_test[i]
        image = image.reshape(1, 28*28)
        predict = _trainer.predict(image)
        predict_label = np.argmax(predict.numpy(), axis=1)[0]
        predicted.append(int(predict_label))
        target.append(label)
    print("Accuracy:", accuracy_score(target, predicted))

Обученая модель показывает точность равную похожей модели обученной при помощи keras (0.9822).

In [44]:
eval(trainer)

Accuracy: 0.9822
